In [1]:
%cd semantic_code_search/code_search/modeling

/teamspace/studios/this_studio/semantic_code_search/code_search/modeling


In [2]:
from omegaconf import OmegaConf
from enum import Enum, auto
from abc import ABC, abstractmethod
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from peft import (PeftModel, PeftConfig, PrefixTuningConfig, 
                 LoraConfig, get_peft_model, TaskType)
from omegaconf import DictConfig
from typing import Optional, Dict, Type
from optimizers import *
from model_manager import *
from losses import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
criterion = InfoNCELoss(temperature=0.07)
bank = MemoryBank(size=1024,embedding_dim=128,device=device)

In [5]:
q = torch.randn(8,128)
k = torch.randn(8,128)

In [6]:
loss,logits = criterion(query_embeddings=q,key_embeddings=k,memory_bank=bank)

In [9]:
logits.shape

torch.Size([8, 1032])

In [5]:
config = OmegaConf.load("/teamspace/studios/this_studio/semantic_code_search/conf/modeling.yaml")

In [70]:
model = Model(config.model.model_id)

In [59]:
model_manager = ModelManager()

In [60]:
model = model_manager.prepare_model(model=model,fine_tuning_type=FineTuningType.FULL,config=config.model.fine_tuning,peft_type=PEFTType.LORA)

In [4]:
torch.arange(10,like=torch.float)

TypeError: arange() received an invalid combination of arguments - got (int, like=torch.dtype), but expected one of:
 * (Number end, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Number start, Number end, *, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Number start, Number end, Number step = 1, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)


In [72]:
num_layers = len([name for name in prepared_model.named_modules() if "encoder.layer" in name])

In [76]:
print([name for name in prepared_model.named_modules() if "encoder.layer" in name])

[('encoder.layer', ModuleList(
  (0-11): 12 x JinaBertLayer(
    (attention): JinaBertAttention(
      (self): JinaBertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (layer_norm_q): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (layer_norm_k): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (output): JinaBertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (layer_norm_1): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (layer_norm_2): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (mlp): JinaBertGLUMLP(
      (act): GELU(approxim

In [46]:

for name, param in model.named_parameters():
    if any(layer in name for layer in trainable_layers):
        assert param.requires_grad == True
    elif "embeddings" in name:
        assert param.requires_grad == False
    elif param.requires_grad:
        assert False

AssertionError: 

In [37]:

for name, param in model.named_parameters():
    if "embeddings" not in name:
        assert param.requires_grad == True
    else:
        assert param.requires_grad == False